In [1]:
import xarray as xr
import numpy as np
from climatico import enso
import matplotlib.pyplot as plt
import cftime
import cartopy
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
from climatico.util import weighted_mean, pacific_lon
import matplotlib.patches as mpatches
import cartopy.feature as cfeature
from shapely.geometry.polygon import Polygon

In [2]:
from config import directory_figs, directory_data

In [37]:
THEDEPTH = 'DEPTH_OF_10C'

In [38]:
def grab_enso_spatial(ds, indices, ds_oni, year1, year2, cutoff, filevar='SST'):
    """
    Use the input seasonal file ('QS-DEC') and precomputed Nino-index to extract Nino(a) 
    event indices in time array of SST file.
    Then extract the variable associated with the Nino(a) events. These focus on DJF months.
    
    Args:
        ds (xarray data array): Seasonal variable to filter for plotting 
                                    (e.g., ds.resample(time='QS-DEC').mean(skipna=True)).
        indices (numpy array): ONI values as numpy array 
                                    (e.g., control_nino.resample(time='QS-DEC').mean(skipna=True).values).
        ds_oni (xarray data array): ONI seasonal xarray 
                                    (e.g., control_nino.resample(time='QS-DEC').mean(skipna=True)).
        year1 (int): First year for range filtering.
        year2 (int): Second year for range filtering.
        cutoff (float): Cutoff of ENSO events. E.g., +/-0.5 for ONI.
        filevar (str): Variable name for input ds. Defaults to ``SST.``
    """
    # do checks
    check1 = ds[filevar].isel(TIME=np.where(indices>=cutoff)[0])[(ds[filevar].isel(TIME=np.where(indices>=cutoff)[0])['TIME'].dt.month==12)].coords['TIME']
    check2 = ds_oni.isel(time=np.where(indices>=cutoff)[0])[(ds_oni.isel(time=np.where(indices>=cutoff)[0])['time'].dt.month==12)].coords['time']
    assert np.all(check1.values==check2.values), "Nino events don't match"
    print("Nino check passed")
    
    check1 = ds[filevar].isel(TIME=np.where(indices<=cutoff)[0])[(ds[filevar].isel(TIME=np.where(indices<=cutoff)[0])['TIME'].dt.month==12)].coords['TIME']
    check2 = ds_oni.isel(time=np.where(indices<=cutoff)[0])[(ds_oni.isel(time=np.where(indices<=cutoff)[0])['time'].dt.month==12)].coords['time']
    assert np.all(check1.values==check2.values), "Nina events don't match"
    print("Nina check passed")
    
    ### nino
    # filter for djf quarters
    sv_indices = ds[filevar].isel(TIME=np.where(indices>=cutoff)[0])[(ds[filevar].isel(TIME=np.where(indices>=cutoff)[0])['TIME'].dt.month==12)]
    # filter for correct year range
    sv_indices = sv_indices[(sv_indices['TIME'].dt.year>=year1)&(sv_indices['TIME'].dt.year<=year2)]
    # take spatial mean for plotting
    nino = sv_indices.mean(dim='TIME')
    print("Nino done")

    ### nina
    # filter for djf quarters
    sv_indices = ds[filevar].isel(TIME=np.where(indices<=cutoff)[0])[(ds[filevar].isel(TIME=np.where(indices<=cutoff)[0])['TIME'].dt.month==12)]
    # filter for correct year range
    sv_indices = sv_indices[(sv_indices['TIME'].dt.year>=year1)&(sv_indices['TIME'].dt.year<=year2)]
    # take spatial mean for plotting
    nina = sv_indices.mean(dim='TIME')
    print("Nina done")
    
    return nino, nina

def pop_lon_nino34():
    """
    Extract mask for the nino-3.4 region. Mask contains ones and nans.
    """
    for_lon = xr.open_dataset('/glade/scratch/molina/amoc_exp/b.e11.B1850LENS.f09_g16.FWAtSalG02Sv.pop.h.SST.000101-005012.nc')
    mask = for_lon['SST'].where((for_lon['TLAT']<5) & (for_lon['TLAT']>-5) & (for_lon['TLONG']>-170+360) & (for_lon['TLONG']<-120+360), 
                                 drop=False).isel(z_t=0, time=0).values
    return np.where(np.isnan(mask), np.nan, 1)

def compute_iso(twodim_array, mask):
    """
    Create array of depth of isotherm using 3d iso array and 2d mask.
    Args:
        twodim_array (numpy array): Isotherm values.
        mask (numpy array): Mask from pop_lon_indx.
    Returns:
        One dimensional array across Pacific slab region.
    """
    temp = (twodim_array[:,:] * mask[:,:]) * 0.01
    temp = np.nanmean(temp, axis=0)
    return temp[~np.isnan(temp)]

In [39]:
# list of filenames to do this for:
# SSTs
file_g02sv = 'b1d.e11.B1850LENS.f09_g16.FWAtSalG02Sv.pop.h.SST.*.nc'
file_g04sv = 'b1d.e11.B1850LENS.f09_g16.FWAtSalG04Sv.pop.h.SST.*.nc'
file_p02sv = 'b1d.e11.B1850LENS.f09_g16.FWAtSalP02Sv.pop.h.SST.*.nc'
file_p04sv = 'b1d.e11.B1850LENS.f09_g16.FWAtSalP04Sv.pop.h.SST.*.nc'
file_psalt = 'b1d.e11.B1850LENS.f09_g16.FWPaSalP04Sv.pop.h.SST.*.nc'
file_cntrl = 'b1d.e11.B1850C5CN.f09_g16.005.pop.h.SST.*.nc'

# grab isotherms
iso_g02sv = xr.open_dataset(
    f'/glade/scratch/molina/amoc_exp/iso20c_FWAtSalG02Sv.nc').sel(
    TIME=slice(cftime.DatetimeNoLeap(200, 1, 1, 0, 0),cftime.DatetimeNoLeap(501, 12, 1, 0, 0)))
iso_g04sv = xr.open_dataset(
    f'/glade/scratch/molina/amoc_exp/iso20c_FWAtSalG04Sv.nc').sel(
    TIME=slice(cftime.DatetimeNoLeap(200, 1, 1, 0, 0),cftime.DatetimeNoLeap(501, 12, 1, 0, 0)))
iso_p02sv = xr.open_dataset(
    f'/glade/scratch/molina/amoc_exp/iso20c_FWAtSalP02Sv.nc').sel(
    TIME=slice(cftime.DatetimeNoLeap(200, 1, 1, 0, 0),cftime.DatetimeNoLeap(501, 12, 1, 0, 0)))
iso_p04sv = xr.open_dataset(
    f'/glade/scratch/molina/amoc_exp/iso20c_FWAtSalP04Sv.nc').sel(
    TIME=slice(cftime.DatetimeNoLeap(200, 1, 1, 0, 0),cftime.DatetimeNoLeap(501, 12, 1, 0, 0)))
iso_psalt = xr.open_dataset(
    f'/glade/scratch/molina/amoc_exp/iso20c_FWPaSalP04Sv.nc').sel(
    TIME=slice(cftime.DatetimeNoLeap(100, 1, 1, 0, 0),cftime.DatetimeNoLeap(251, 12, 1, 0, 0)))
iso_cntrl = xr.open_dataset(
    f'/glade/scratch/molina/amoc_exp/iso20c_005.nc').sel(
    TIME=slice(cftime.DatetimeNoLeap(800, 1, 1, 0, 0),cftime.DatetimeNoLeap(1600, 12, 1, 0, 0)))


In [40]:
iso_cntrl

<xarray.Dataset>
Dimensions:       (NLAT: 384, NLON: 320, TIME: 9612, bnds: 2)
Coordinates:
  * TIME          (TIME) object 0800-01-01 00:00:00 ... 1600-12-01 00:00:00
Dimensions without coordinates: NLAT, NLON, bnds
Data variables:
    TIME_bnds     (TIME, bnds) object ...
    DEPTH_OF_20C  (TIME, NLAT, NLON) float64 ...
    DEPTH_OF_18C  (TIME, NLAT, NLON) float64 ...
    DEPTH_OF_15C  (TIME, NLAT, NLON) float64 ...
    DEPTH_OF_12C  (TIME, NLAT, NLON) float64 ...
    DEPTH_OF_10C  (TIME, NLAT, NLON) float64 ...
    DEPTH_OF_8C   (TIME, NLAT, NLON) float64 ...
    DEPTH_OF_5C   (TIME, NLAT, NLON) float64 ...
Attributes:
    history:      PyFerret V7.5 (optimized) 17-Feb-21
    Conventions:  CF-1.6

In [41]:
nino = enso.DefineNino(nino='nino34', lats='lat', lons='lon', cutoff=0.5, runningmean=3)

In [42]:
####################################################################

ds2 = xr.open_mfdataset(f'{directory_data}{file_cntrl}',
                        combine='by_coords',
                        preprocess=nino.nino)

# reduce dims to time, lat, lon
ds2 = ds2.isel(z_t=0)

print("indices started")
# compute indices
control_nino = nino.compute_index(ds2['SST'].groupby('time.month'), 
                                  ds2['SST'].groupby('time.month').mean(skipna=True), 
                                  linear_detrend=False, lat_name='lat')

control_nino = control_nino.sel(time=slice(iso_cntrl.TIME.values[0], iso_cntrl.TIME.values[-1]))

# grab numpy array
control_index = control_nino.resample(time='QS-DEC').mean(skipna=True).values
print("indices done")

#################################################################### TEMP

controlnino, controlnina = grab_enso_spatial(
                                              ds = iso_cntrl.resample(TIME='QS-DEC').mean(skipna=True), 
                                              indices = control_index, 
                                              ds_oni = control_nino.resample(time='QS-DEC').mean(skipna=True),
                                              year1 = 801, 
                                              year2 = 1599,
                                              cutoff = nino.cutoff, 
                                              filevar = THEDEPTH)

####################################################################

indices started


/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/xarray/core/indexing.py:1361: PerformanceWarning: Slicing with an out-of-order index is generating 101 times more chunks
  return self.array[key]


indices done
Nino check passed
Nina check passed
Nino done
Nina done


In [43]:
####################################################################

ds1 = xr.open_mfdataset(f'{directory_data}{file_g02sv}',
                        combine='by_coords',
                        preprocess=nino.nino)

# reduce dims to time, lat, lon
ds1 = ds1.isel(z_t=0)

print("indices started")
# compute indices
g02sv_nino = nino.compute_index(ds1['SST'].groupby('time.month'), 
                                ds2['SST'].groupby('time.month').mean(skipna=True), 
                                linear_detrend=False, lat_name='lat')

g02sv_nino = g02sv_nino.sel(time=slice(iso_g02sv.TIME.values[0], iso_g02sv.TIME.values[-1]))

# grab numpy array
g02sv_index = g02sv_nino.resample(time='QS-DEC').mean(skipna=True).values
print("indices done")

#################################################################### TEMP

g02svnino, g02svnina = grab_enso_spatial(
                                          ds = iso_g02sv.resample(TIME='QS-DEC').mean(skipna=True), 
                                          indices = g02sv_index, 
                                          ds_oni = g02sv_nino.resample(time='QS-DEC').mean(skipna=True),
                                          year1 = 201, 
                                          year2 = 500, 
                                          cutoff = nino.cutoff, 
                                          filevar = THEDEPTH)

####################################################################

indices started


/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/xarray/core/indexing.py:1361: PerformanceWarning: Slicing with an out-of-order index is generating 51 times more chunks
  return self.array[key]


indices done
Nino check passed
Nina check passed
Nino done
Nina done


In [44]:
####################################################################

ds1 = xr.open_mfdataset(f'{directory_data}{file_g04sv}',
                        combine='by_coords',
                        preprocess=nino.nino)

# reduce dims to time, lat, lon
ds1 = ds1.isel(z_t=0)

print("indices started")
# compute indices
g04sv_nino = nino.compute_index(ds1['SST'].groupby('time.month'), 
                                ds2['SST'].groupby('time.month').mean(skipna=True), 
                                linear_detrend=False, lat_name='lat')

g04sv_nino = g04sv_nino.sel(time=slice(iso_g04sv.TIME.values[0], iso_g04sv.TIME.values[-1]))

# grab numpy array
g04sv_index = g04sv_nino.resample(time='QS-DEC').mean(skipna=True).values
print("indices done")

#################################################################### TEMP

g04svnino, g04svnina = grab_enso_spatial(
                                          ds = iso_g04sv.resample(TIME='QS-DEC').mean(skipna=True), 
                                          indices = g04sv_index, 
                                          ds_oni = g04sv_nino.resample(time='QS-DEC').mean(skipna=True),
                                          year1 = 201, 
                                          year2 = 500, 
                                          cutoff = nino.cutoff, 
                                          filevar = THEDEPTH)

####################################################################

indices started


/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/xarray/core/indexing.py:1361: PerformanceWarning: Slicing with an out-of-order index is generating 50 times more chunks
  return self.array[key]


indices done
Nino check passed
Nina check passed
Nino done
Nina done


In [45]:
####################################################################

ds1 = xr.open_mfdataset(f'{directory_data}{file_p02sv}',
                        combine='by_coords',
                        preprocess=nino.nino)

# reduce dims to time, lat, lon
ds1 = ds1.isel(z_t=0)

print("indices started")
# compute indices
p02sv_nino = nino.compute_index(ds1['SST'].groupby('time.month'), 
                                ds2['SST'].groupby('time.month').mean(skipna=True), 
                                linear_detrend=False, lat_name='lat')

p02sv_nino = p02sv_nino.sel(time=slice(iso_p02sv.TIME.values[0], iso_p02sv.TIME.values[-1]))

# grab numpy array
p02sv_index = p02sv_nino.resample(time='QS-DEC').mean(skipna=True).values
print("indices done")

#################################################################### TEMP

p02svnino, p02svnina = grab_enso_spatial(
                                          ds = iso_p02sv.resample(TIME='QS-DEC').mean(skipna=True), 
                                          indices = p02sv_index, 
                                          ds_oni = p02sv_nino.resample(time='QS-DEC').mean(skipna=True),
                                          year1 = 201, 
                                          year2 = 500, 
                                          cutoff = nino.cutoff, 
                                          filevar = THEDEPTH)

####################################################################

indices started


/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/xarray/core/indexing.py:1361: PerformanceWarning: Slicing with an out-of-order index is generating 50 times more chunks
  return self.array[key]


indices done
Nino check passed
Nina check passed
Nino done
Nina done


In [46]:
####################################################################

ds1 = xr.open_mfdataset(f'{directory_data}{file_p04sv}',
                        combine='by_coords',
                        preprocess=nino.nino)

# reduce dims to time, lat, lon
ds1 = ds1.isel(z_t=0)

print("indices started")
# compute indices
p04sv_nino = nino.compute_index(ds1['SST'].groupby('time.month'), 
                                ds2['SST'].groupby('time.month').mean(skipna=True), 
                                linear_detrend=False, lat_name='lat')

p04sv_nino = p04sv_nino.sel(time=slice(iso_p04sv.TIME.values[0], iso_p04sv.TIME.values[-1]))

# grab numpy array
p04sv_index = p04sv_nino.resample(time='QS-DEC').mean(skipna=True).values
print("indices done")

#################################################################### TEMP

p04svnino, p04svnina = grab_enso_spatial(
                                          ds = iso_p04sv.resample(TIME='QS-DEC').mean(skipna=True), 
                                          indices = p04sv_index, 
                                          ds_oni = p04sv_nino.resample(time='QS-DEC').mean(skipna=True),
                                          year1 = 201, 
                                          year2 = 500, 
                                          cutoff = nino.cutoff, 
                                          filevar = THEDEPTH)

####################################################################

indices started


/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/xarray/core/indexing.py:1361: PerformanceWarning: Slicing with an out-of-order index is generating 50 times more chunks
  return self.array[key]


indices done
Nino check passed
Nina check passed
Nino done
Nina done


In [47]:
####################################################################

ds1 = xr.open_mfdataset(f'{directory_data}{file_psalt}',
                        combine='by_coords',
                        preprocess=nino.nino)

# reduce dims to time, lat, lon
ds1 = ds1.isel(z_t=0)

print("indices started")
# compute indices
psalt_nino = nino.compute_index(ds1['SST'].groupby('time.month'), 
                                ds2['SST'].groupby('time.month').mean(skipna=True), 
                                linear_detrend=False, lat_name='lat')

psalt_nino = psalt_nino.sel(time=slice(iso_psalt.TIME.values[0], iso_psalt.TIME.values[-1]))

# grab numpy array
psalt_index = psalt_nino.resample(time='QS-DEC').mean(skipna=True).values
print("indices done")

#################################################################### TEMP

psaltnino, psaltnina = grab_enso_spatial(
                                          ds = iso_psalt.resample(TIME='QS-DEC').mean(skipna=True), 
                                          indices = psalt_index, 
                                          ds_oni = psalt_nino.resample(time='QS-DEC').mean(skipna=True),
                                          year1 = 101, 
                                          year2 = 250, 
                                          cutoff = nino.cutoff, 
                                          filevar = THEDEPTH)

####################################################################

indices started


/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/xarray/core/indexing.py:1361: PerformanceWarning: Slicing with an out-of-order index is generating 50 times more chunks
  return self.array[key]


indices done
Nino check passed
Nina check passed
Nino done
Nina done


In [48]:
poplon = pop_lon_nino34()

cntrl_diff_nino = compute_iso(controlnino.values, poplon)
g02sv_diff_nino = compute_iso(g02svnino.values, poplon)
g04sv_diff_nino = compute_iso(g04svnino.values, poplon)
p02sv_diff_nino = compute_iso(p02svnino.values, poplon)
p04sv_diff_nino = compute_iso(p04svnino.values, poplon)
psalt_diff_nino = compute_iso(psaltnino.values, poplon)

/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/ipykernel_launcher.py:69: RuntimeWarning: Mean of empty slice


In [49]:
data_assemble=xr.Dataset({
                 'cntrl_nino':(['a'], cntrl_diff_nino),
                 'g02sv_nino':(['a'], g02sv_diff_nino),
                 'g04sv_nino':(['a'], g04sv_diff_nino),
                 'p02sv_nino':(['a'], p02sv_diff_nino),
                 'p04sv_nino':(['a'], p04sv_diff_nino),
                 'psalt_nino':(['a'], psalt_diff_nino)})

In [50]:
data_assemble.to_netcdf(f'{directory_data}ninoslabs_{THEDEPTH}.nc')

In [51]:
cntrl_diff_nina = compute_iso(controlnina.values, poplon)
g02sv_diff_nina = compute_iso(g02svnina.values, poplon)
g04sv_diff_nina = compute_iso(g04svnina.values, poplon)
p02sv_diff_nina = compute_iso(p02svnina.values, poplon)
p04sv_diff_nina = compute_iso(p04svnina.values, poplon)
psalt_diff_nina = compute_iso(psaltnina.values, poplon)

/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/ipykernel_launcher.py:69: RuntimeWarning: Mean of empty slice


In [52]:
data_assemble=xr.Dataset({
                 'cntrl_nina':(['a'], cntrl_diff_nina),
                 'g02sv_nina':(['a'], g02sv_diff_nina),
                 'g04sv_nina':(['a'], g04sv_diff_nina),
                 'p02sv_nina':(['a'], p02sv_diff_nina),
                 'p04sv_nina':(['a'], p04sv_diff_nina),
                 'psalt_nina':(['a'], psalt_diff_nina)})

In [53]:
data_assemble.to_netcdf(f'{directory_data}ninaslabs_{THEDEPTH}.nc')